This notebook analyzes data from https://github.com/HuthLab/deep-fMRI-dataset. To set up, see instructions in the `deep-fMRI-dataset` folder.

In [1]:
%load_ext autoreload
%autoreload 2
import datasets
import numpy as np
from os.path import join
# from ridge_utils.SemanticModel import SemanticModel
from matplotlib import pyplot as plt
from typing import List
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer
# from feature_spaces import em_data_dir, data_dir, results_dir
from collections import defaultdict
import pandas as pd
import os
import seaborn as sns
# from fit_linear_models import get_dsets
# from ridge_utils.SemanticModel import SemanticModel
# import encoding_utils, feature_spaces
from transformers.pipelines.pt_utils import KeyDataset
# from feature_spaces import *
from transformers import pipeline
import datasets

In [2]:
dset = datasets.load_dataset('demelin/moral_stories', 'cls-action+norm-minimal_pairs')

/home/chansingh/.embgam/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Found cached dataset moral_stories (/home/chansingh/.cache/huggingface/datasets/demelin___moral_stories/cls-action+norm-minimal_pairs/1.1.0/47de080a74d44a1d4785a2c16fe6c7a978ef218fb0dc319d8392d22337f7b806)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dset.shape

{'train': (20000, 9), 'validation': (2000, 9), 'test': (2000, 9)}

In [4]:
d = dset['train']

In [6]:
text.shape

(20000, 2)

# Test models

In [15]:
# glove = SemanticModel.load_np(join(nlp_utils_dir, 'glove'))
wordseqs = get_story_wordseqs(['sloth'])
# vectors = {}
# for story in allstories:
	# sm = apply_model_to_words(wordseqs[story], glove, 300)
	# vectors[story] = sm.data
# return downsample_word_vectors(allstories, vectors, wordseqs)

In [16]:
ds = wordseqs['sloth']

In [17]:
pipe = pipeline("feature-extraction", model="bert-base-uncased", device=0)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
ngram_size = 9
ngrams_list = []
for i in range(100):
    l = max(0, i + 1 - ngram_size)
    ngram = ' '.join(ds.data[l: i])
    ngrams_list.append(ngram)

In [21]:
dset = datasets.Dataset.from_dict({'text':ngrams_list})

In [30]:
dset

Dataset({
    features: ['text'],
    num_rows: 100
})

In [32]:
def extract(x):
    return {'emb': pipe(x['text'])}

In [35]:
d2 = dset.map(extract)

  0%|          | 0/100 [00:00<?, ?ex/s]

/home/chansingh/.embgam/lib/python3.8/site-packages/transformers/pipelines/base.py:976: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [40]:
out_list = d2['emb']

In [44]:
len(out_list[0][0][0])

768

In [45]:


# print(ngrams_list[:10])
# embeddings are (batch_size, 1, (seq_len + 2), 768) -- BERT adds initial / final tokens
# out_list = pipe(ngrams_list)

# convert to np array by averaging over len (can't just convert this since seq lens vary)
# embs = np.array(out).squeeze().mean(axis=1)
num_ngrams = len(out_list)
dim_size = len(out_list[0][0][0])
embs = np.zeros((num_ngrams, dim_size))
for i in range(len(out_list)):
    embs[i] = np.mean(out_list[i], axis=1) # avg over seq_len dim